Обучение моделей

In [51]:
import numpy 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from skl2onnx import to_onnx
import onnxruntime as rt

In [46]:
iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
onx = to_onnx(clr, X[:1])
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

y_pred_train=model.predict_proba(X_test)
numpy.save(f'{model_name}_predict.npy',y_pred_train)

In [5]:
# Compute the prediction with onnxruntime.
import onnxruntime as rt

sess = rt.InferenceSession("rf_iris.onnx", providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]

In [12]:
sess.get_modelmeta()

catboost

In [67]:
import catboost


model = catboost.CatBoostClassifier(loss_function='MultiClass')

model.fit(X_train, y_train)
# Save model to ONNX-ML format
model.save_model(
    "CatB.onnx",
    format="onnx",
    pool=X[:1]
)

model_name='CatB'
y_pred_train=model.predict_proba(X_test)
numpy.save(f'{model_name}_predict.npy',y_pred_train)

Learning rate set to 0.070767
0:	learn: 1.0179915	total: 1.17ms	remaining: 1.16s
1:	learn: 0.9509828	total: 2.02ms	remaining: 1.01s
2:	learn: 0.9004688	total: 2.96ms	remaining: 983ms
3:	learn: 0.8479588	total: 3.87ms	remaining: 964ms
4:	learn: 0.8022598	total: 4.76ms	remaining: 947ms
5:	learn: 0.7509315	total: 5.66ms	remaining: 937ms
6:	learn: 0.7179107	total: 6.03ms	remaining: 855ms
7:	learn: 0.6800420	total: 7.05ms	remaining: 874ms
8:	learn: 0.6463904	total: 7.74ms	remaining: 852ms
9:	learn: 0.6164655	total: 8.83ms	remaining: 874ms
10:	learn: 0.5833592	total: 10.1ms	remaining: 910ms
11:	learn: 0.5563356	total: 16.7ms	remaining: 1.37s
12:	learn: 0.5313970	total: 19.3ms	remaining: 1.47s
13:	learn: 0.5114578	total: 22.2ms	remaining: 1.56s
14:	learn: 0.4896886	total: 25.2ms	remaining: 1.66s
15:	learn: 0.4696421	total: 28.7ms	remaining: 1.76s
16:	learn: 0.4510253	total: 31.5ms	remaining: 1.82s
17:	learn: 0.4364347	total: 33.4ms	remaining: 1.82s
18:	learn: 0.4201299	total: 35.3ms	remaining

In [33]:
sess = rt.InferenceSession("CatB.onnx", providers=["CPUExecutionProvider"])
len(sess.get_inputs()),len(sess.get_outputs())
print([nd.name for nd in sess.get_inputs()])
print([nd.name for nd in sess.get_outputs()])

['features']
['label', 'probabilities']


In [34]:
pred_onx = sess.run(['probabilities'], {'features': X_test})[0]

lightgbm

In [48]:
import lightgbm as lgb
from onnxmltools.convert import convert_lightgbm
from skl2onnx.common.data_types import FloatTensorType

In [49]:
model=lgb.LGBMClassifier()
model.fit(X_train, y_train)

LGBMClassifier()

In [52]:
onnx_model = convert_lightgbm(model, initial_types=[('X', FloatTensorType(shape=[None,X_train.shape[1]]))],target_opset=9)
with open("lgb.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

#FloatTensorType (shape)
# shape указывает какой тип данных в указанных строках и стбл
# None означает что мы не знаем количество входных строк

model_name='lgb'
y_pred_train=model.predict_proba(X_test)
numpy.save(f'{model_name}_predict.npy',y_pred_train)

In [15]:
sess = rt.InferenceSession("lgb.onnx", providers=["CPUExecutionProvider"])
len(sess.get_inputs()),len(sess.get_outputs())
print([nd.name for nd in sess.get_inputs()])
print([nd.name for nd in sess.get_outputs()])

['X']
['label', 'probabilities']


In [36]:
y_test_pred_onnx=np.array([row[2] for row in sess.run(output_names=['probabilities'],input_feed={'X':X_test})[0]])
y_test_pred=np.array([row[2] for row in model.predict_proba(X_test)])

In [38]:
np.abs((y_test_pred-y_test_pred_onnx)/y_test_pred).max()

1.4901116811865072e-06

xgboost

In [53]:
from xgboost import XGBClassifier
from onnxmltools.convert import convert_xgboost

In [54]:
model=XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [55]:
onnx_model = convert_xgboost(model, initial_types=[('X', FloatTensorType(shape=[None,X_train.shape[1]]))])
with open("xgb.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

model_name='xgb'
y_pred_train=model.predict_proba(X_test)
numpy.save(f'{model_name}_predict.npy',y_pred_train)

In [38]:
sess = rt.InferenceSession("xgb.onnx", providers=["CPUExecutionProvider"])
print(len(sess.get_inputs()),len(sess.get_outputs()))
print([nd.name for nd in sess.get_inputs()])
print([nd.name for nd in sess.get_outputs()])

1 2
['X']
['label', 'probabilities']


In [40]:
y_test_pred_onnx=np.array([row[2] for row in sess.run(output_names=['probabilities'],input_feed={'X':X_test})[0]])
y_test_pred=np.array([row[2] for row in model.predict_proba(X_test)])
np.abs((y_test_pred-y_test_pred_onnx)/y_test_pred).max()

1.9163115e-06

In [44]:
numpy.save('X_test.npy',X_test)

In [65]:
sess.get_inputs()[0].name

'features'

In [66]:
X_test=numpy.load('X_test.npy')
for model_name in ['xgb','CatB','lgb']:
    sess = rt.InferenceSession(f"{model_name}.onnx", providers=["CPUExecutionProvider"])
    y_test_pred_onnx=np.array([row[2] for row in sess.run(output_names=[],input_feed={sess.get_inputs()[0].name:X_test})[1]])
    y_test_pred=numpy.array([row[2] for row in numpy.load(f'{model_name}_predict.npy')])
    print(model_name)
    print(np.abs((y_test_pred-y_test_pred_onnx)/y_test_pred).max())
    print('')

xgb
1.4203836e-06

CatB
9.549157132711615e-06

lgb
2.081079999869954e-06



In [70]:
X_test=numpy.load('X_test.npy')
for model_name in ['CatB']:
    sess = rt.InferenceSession(f"{model_name}.onnx", providers=["CPUExecutionProvider"])
    y_test_pred_onnx=np.array([row[2] for row in sess.run(output_names=[],input_feed={sess.get_inputs()[0].name:X_test})[1]])
    y_test_pred=numpy.array([row[2] for row in numpy.load(f'{model_name}_predict.npy')])
    print(model_name)
    print(np.abs((y_test_pred-y_test_pred_onnx)/y_test_pred).max())
    print('')

CatB
9.549157132711615e-06



Тестируем виртуальный контейнер

In [1]:
from io import BytesIO
import onnxruntime as rt

C:\Users\1\Desktop\ONNX\dev_env\Lib\site-packages\onnxruntime\capi\_pybind_state.py:26: UserWarning: Please install the 2019 Visual C++ runtime and then try again. If you've installed the runtime in a non-standard location (other than %SystemRoot%\System32), make sure it can be found by setting the correct path.
  warnings.warn("Please install the 2019 Visual C++ runtime and then try again. "


In [5]:
import onnx

In [25]:
# model=onnx.load('lgb.onnx')
# type(model)

In [4]:
f=open('lgb.onnx','rb').read()
mybyte=BytesIO(f.read())
mybyte.seek(0)
arr=list(mybyte.read())

In [27]:
arr2model=BytesIO()
arr2model.write(bytes(arr))
arr2model.seek(0)
model=onnx.load(arr2model)

In [30]:
arr2model

In [8]:
f=open('lgb.onnx','rb').read()
sess = rt.InferenceSession(f, providers=["CPUExecutionProvider"])

In [12]:
sess.get_inputs()[0].name

'X'

In [39]:
f.close()